<a href="https://colab.research.google.com/github/SuzuT1212/hokudai_sylbs/blob/main_suzu/%E3%82%B7%E3%83%A9%E3%83%90%E3%82%B9%E3%82%B9%E3%82%AF%E3%83%AC%E3%82%A4%E3%83%94%E3%83%B3%E3%82%B0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!apt-get update
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin
!pip install selenium

Ign:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:2 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:3 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:4 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Get:5 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Ign:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [697 B]
Hit:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:9 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Get:10 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:11 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:12 http://archive.ubuntu.com/ubuntu bionic-backpor

driveをマウントする

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#SeleniumとBeautifulSoupのライブラリをインポート
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np


Chromeからサイトを開く

In [ ]:
options = webdriver.ChromeOptions()
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')
driver = webdriver.Chrome('chromedriver',options=options)
driver.implicitly_wait(10)
driver.get("https://syllabus01.academic.hokudai.ac.jp/Syllabi/Public/Syllabus/SylSearch.aspx")
html = driver.page_source.encode('utf-8')
soup = BeautifulSoup(html, "html.parser")


タブを選択

In [ ]:
from selenium.webdriver.support.ui import Select
# 年度の指定
year_element = driver.find_element_by_id('ctl00_phContents_ucSylSearchuc_ddl_year')
year_select_element = Select(year_element)
year_select_element.select_by_value('2020')

# 課程区分の指定
degree_element = driver.find_element_by_id('ctl00_phContents_ucSylSearchuc_ddl_org')
degree_select_element = Select(degree_element)
degree_select_element.select_by_value('02')

# 開講学部の指定
fac_element = driver.find_element_by_id('ctl00_phContents_ucSylSearchuc_ddl_fac')
fac_select_element = Select(fac_element)
fac_select_element.select_by_value('00')

# 学期の指定
semester_element = driver.find_element_by_id('ctl00_phContents_ucSylSearchuc_ddl_open')
semester_select_element = Select(semester_element)
semester_select_element.select_by_value('1')

# 検索をクリック
driver.find_element_by_xpath('//*[@id="ctl00_phContents_ucSylSearchuc_ctl109_btnSearch"]').click()

# 表示件数を全件にする
exp_element = driver.find_element_by_id('ctl00_phContents_ucSylList_DDLLine_ddl')
exp_select_element = Select(exp_element)
exp_select_element.select_by_value('0')


スクレイピング

In [ ]:
# htmlを取得しスクレイピングしていく
html = driver.page_source.encode('utf-8')
soup = BeautifulSoup(html, "html.parser")
# テーブルを指定
table = soup.find('table', id="ctl00_phContents_ucSylList_gv").tbody
rows = table.find_all('tr')
# print(rows[1])


In [ ]:
# dateframeを作るためにcolumn作成
columns = [v.text for v in rows[0].find_all('th')]
# ['\xa0', '期間Semester', '科目名Course Title', '担当教員Teaching staff in charge', '曜日・時限Day of the week, period', 
# '対象年次Year of Eligible Students', '実務経験のある教員等による授業科目Course list by the instructor with practical experiences', '\xa0']

# del columns[0]
# del columns[-1]

# ['期間Semester', '科目名Course Title', '担当教員Teaching staff in charge', '曜日・時限Day of the week, period', 
# '対象年次Year of Eligible Students', '実務経験のある教員等による授業科目Course list by the instructor with practical experiences']

df = pd.DataFrame(columns=columns)

for i in range(len(rows)):
  # 全ての<td>タグ（セルデータ）を取得しtdsに格納、リスト化
  tds = rows[i].find_all('td')
  # tdsのデータ数がカラム数に一致しない場合（ブランク）などは排除し、
  if len(tds) == len(columns):
    # （ある行成分の）全セルデータをテキスト成分としてvaluesに格納、リスト化
    values = [ td.text.replace('\n', '').replace('\xa0', ' ') for td in tds ]
    # valuesをpd.seriesデータに変換、データフレームに結合
    df = df.append(pd.Series(values, index=columns), ignore_index= True)
df.head()


,,期間Semester,科目名Course Title,担当教員Teaching staff in charge,"曜日・時限Day of the week, period",対象年次Year of Eligible Students,実務経験のある教員等による授業科目Course list by the instructor with practical experiences,
0,1 ...,１学期1st Semester,一般教育演習(ﾌﾚｯｼｭﾏﾝｾﾐﾅｰ)戦後日本史Freshman SeminarHistor...,白木沢 旭児 ( 大学院文学研究院 )SHIRAKIZAWA Asahiko ( Facul...,火5Tue.5,1～,該当しないN/A,
1,2 ...,１学期1st Semester,一般教育演習(ﾌﾚｯｼｭﾏﾝｾﾐﾅｰ)格差の社会学入門Freshman SeminarSoc...,平澤 和司 ( 大学院文学研究院 )HIRASAWA Kazushi ( Faculty o...,火5Tue.5,1～,該当しないN/A,
2,3 ...,１学期1st Semester,一般教育演習(ﾌﾚｯｼｭﾏﾝｾﾐﾅｰ)トピックス生物学Freshman SeminarCur...,,集中Intensive,1～,該当しないN/A,
3,4 ...,１学期1st Semester,一般教育演習(ﾌﾚｯｼｭﾏﾝｾﾐﾅｰ)マイコン制御ロボットをつくるFreshman Semi...,田中 孝之 ( 大学院情報科学研究院 )TANAKA Takayuki ( Faculty ...,火5Tue.5,1～,該当しないN/A,
4,5 ...,１学期1st Semester,一般教育演習(ﾌﾚｯｼｭﾏﾝｾﾐﾅｰ)森林科学への招待Freshman SeminarInv...,澤田 圭 ( 大学院農学研究院 )SAWATA Kei ( Research Faculty...,火5Tue.5,1～,該当しないN/A,


Datebaseを作成する

In [ ]:
import sqlite3
from sqlalchemy import create_engine

engine = create_engine('sqlite:///syllbs1.db', echo=False,encoding="utf-8")
df.to_sql("df", engine)